#### Incremental data ingestion using AutoLoader

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
dbutils.widgets.text("ticker", "")
tick = dbutils.widgets.get("ticker")

In [0]:
expected_schema = StructType([
    StructField("date", StringType(), True),
    StructField("open", DoubleType(), True),
    StructField("high", DoubleType(), True),
    StructField("low", DoubleType(), True),
    StructField("close", DoubleType(), True),
    StructField("adj_close", DoubleType(), True),
    StructField("volume", LongType(), True)
])

In [ ]:
df = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "csv")\
    .option("cloudFiles.schemaLocation", f"/Volumes/stock_data_cata/bronze/bronzevolume/{tick}/checkpoint")\
    .option("header", "true")\
    .schema(expected_schema)\
    .load(f"/Volumes/stock_data_cata/raw/rawvolume/{tick}/")

In [ ]:
df.writeStream.format("delta")\
    .option("checkpointLocation", f"/Volumes/stock_data_cata/bronze/bronzevolume/{tick}/checkpoint")\
    .option("path", f"/Volumes/stock_data_cata/bronze/bronzevolume/{tick}/batch_data")\
    .trigger(once=True)\
    .outputMode("append")\
    .start()